# Questions 2

This script compares the approaches of upsampling then filtering versus applying a polyphase upsampling filter.

In [ ]:
from pathlib import Path

import numpy as np
import scipy.fft as fft
import scipy.signal as signal

import matplotlib.pyplot as plt
import seaborn as sns

from a3_config import A3_ROOT, SAVEFIG_CONFIG

Define filter specifications:

In [ ]:
FS     = 0.5     # sampling frequency, kHz
F_PASS = 0.2    # cutoff frequency, kHz
F_STOP = 0.3    # stop band frequency, kHz
A_PASS = 3      # pass band attenuation, dB
A_STOP = 100    # stop band attenuation, dB

### Upsample

In [ ]:
# Import polyphase downsampled signal from Question 1
t_signal, x_signal = np.load(Path(A3_ROOT, "output", "q1_signal_out.npy"))

In [ ]:
M = 80 # determined in Question 1
x_zpack = np.concatenate([[x]+[0]*(M-1) for x in x_signal])
h_zpack = fft.fft(x_zpack, 8192)[:4096]

t_zpack = np.arange(0, 50, 1 / (FS * M))
f_zpack = fft.fftfreq(8192, 1 / (FS * M))[:4096] * 1000 # show in Hz rather than kHz

fig, axs = plt.subplots(2, figsize=(6, 3))
fig.tight_layout()

sns.lineplot(x=t_zpack, y=x_zpack.real, ax=axs[0])
sns.lineplot(x=f_zpack, y=np.abs(h_zpack), ax=axs[1])

axs[0].set_xlabel("Time (ms)")
axs[1].set_xlabel("Frequency (Hz)")
axs[1].set_xlim([-13, 263])

# fname = Path(A3_ROOT, "output", "q2_zpack.png")
# fig.savefig(fname, **SAVEFIG_CONFIG)
plt.show()

### Attempt Sinc Interpolation

In [ ]:
# Apply rectangular window in freq. domain (i.e. sinc interpolation)
N_zpack = len(x_zpack); passband = int(np.ceil(N_zpack * 0.005)) + 1
f_mask1 = np.ones(len(x_zpack)); f_mask1[passband:-passband] = 0
x_usamp = fft.ifft(fft.fft(x_zpack) * f_mask1)

passband = int(np.ceil(4096 * 0.01)) + 1
f_mask2 = np.ones(4096); f_mask2[passband:] = 0
h_usamp = fft.fft(x_zpack, 8192)[:4096] * f_mask2

fig, axs = plt.subplots(2, figsize=(6, 3))
fig.tight_layout()

sns.lineplot(x=t_zpack, y=x_usamp.real, ax=axs[0])
sns.lineplot(x=f_zpack, y=np.abs(h_usamp), ax=axs[1])

axs[0].set_xlabel("Time (ms)")
axs[1].set_xlabel("Frequency (Hz)")
axs[1].set_xlim([-13, 263])

# fname = Path(A3_ROOT, "output", "q2_usamp.png")
# fig.savefig(fname, **SAVEFIG_CONFIG)
plt.show()

### Attempt Kaiser LPF

In [ ]:
ripple_p = 1 - np.power(10, -A_PASS / 20)
ripple_s = np.power(10, -A_STOP / 20)
print("Maximum pass band ripple:", ripple_p)
print("Maximum stop band ripple:", ripple_s)

A = -20 * np.log10(min(ripple_p, ripple_s))
print("Required attenuation:", A, "dB")

In [ ]:
# Kaiser window filter length estimate
N = int(np.ceil((A - 7.95)/(14.36 * ((F_STOP - F_PASS) / (FS * 80)))))
N = N + 1 if (N % 2) else N
print("Filter length estimate:", N)

beta = 0.1102 * (A - 8.7)
print("Kaiser window beta:", beta)

Construct a vector representing the ideal frequency response.

In [ ]:
# Calculate pass band width, L
L = int(np.round(N * F_PASS / (FS * 80)))
print("Bins in passband:", L)

# Construct V, with 1's in the pass band and 0's in the stop band
h_ideal = np.zeros(N//2)
h_ideal[:L] = np.ones(L)
h_ideal = np.concatenate([h_ideal, np.flip(h_ideal)])

# Construct a frequency axis for plotting
f_ideal = np.linspace(0, FS * 80, N)

# Plot ideal frequency response, represented by vector V
fig, ax = plt.subplots(figsize=(6, 2))
fig.tight_layout()

sns.lineplot(x=f_ideal[:N//2], y=h_ideal[:N//2], ax=ax)
ax.set_xlim([0, 1])
ax.set_xlabel("Frequency (kHz)")
ax.set_ylabel("Gain")

plt.show()

In [ ]:
# Helper function for converting frequency response to dB scale
dB = lambda x: 20 * np.log10(x)

def plot_freqz(w, h, fname=None):
    """Plot frequency response and overlay filter requirements."""
    fig, ax = plt.subplots(figsize=(6, 3))
    fig.tight_layout()
    sns.lineplot(x=w, y=dB(np.abs(h)), ax=ax)
    # Axis labels
    ax.set_xlabel("Frequency (kHz)")
    ax.set_ylabel("Gain (dB)")
    ax.legend(framealpha=1)
    # Save or just show
    if fname:
        fig.savefig(Path(A3_ROOT, "output", fname), **SAVEFIG_CONFIG)
    plt.show()

In [ ]:
# Impulse (time) response of ideal filter
x_ideal = fft.fftshift(fft.ifft(h_ideal))

# Construct and apply the Kaiser window
x_windowed = x_ideal * signal.windows.kaiser(N, beta)
h_windowed = fft.fft(x_windowed, 512)[:256]

# Construct frequency axis for plotting
f_windowed = np.linspace(0, FS * 80 / 2, 256)

plot_freqz(f_windowed, h_windowed)

In [ ]:
# Apply filter to signal, removing transient edge effects
x_filt = signal.convolve(x_windowed, x_zpack)[N//2:-(N//2-1)]
h_filt = fft.fft(x_filt, 8192)[:4096]

fig, axs = plt.subplots(2, figsize=(6, 3))
fig.tight_layout()

sns.lineplot(x=t_zpack, y=x_filt.real, ax=axs[0])
sns.lineplot(x=f_zpack, y=np.abs(h_filt), ax=axs[1])

axs[0].set_xlabel("Time (ms)")
axs[1].set_xlabel("Frequency (Hz)")
axs[1].set_xlim([-13, 263])

# fname = Path(A3_ROOT, "output", "q1_filt.png")
# fig.savefig(fname, **SAVEFIG_CONFIG)
plt.show()

### Polyphase Upsample

In [ ]:
# Reshape filter coefficients into matrix, zero padded to multiple of M
k = M - (N % M)
polyfilt = np.concatenate([x_windowed, np.zeros(k)])
polyfilt = polyfilt.reshape(int((N + k) / M), M).T  # reshape row-major then transpose
polyfilt = np.flipud(polyfilt)                      # vertical flip

In [ ]:
# Concatenate results into output array, which becomes the filtered signal
# x_polyfilt = np.empty(0, dtype=np.complex128)
# for i in range(M):
#     x_polyfilt = np.append(x_polyfilt, signal.convolve(polyfilt[i], x_signal))
x_polyfilt = []
for x in x_signal:
    for i in range(M):
        x_polyfilt.append(sum(x * b for b in polyfilt[i])) # TODO: this is clearly incorrect...

x_polyfilt = np.array(x_polyfilt)
plt.plot(x_polyfilt)